# CRF based NER
- [참고 코드](https://lovit.github.io/nlp/2018/06/22/crf_based_ner/)
- data : CoNLL 2002 dataset

# 0. Settings

In [9]:
!pip install python-crfsuite

     |████████████████████████████████| 186 kB 1.8 MB/s eta 0:00:01


In [90]:
import nltk
import pycrfsuite
import warnings
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from itertools import chain
#warnings.filterwarnings('ignore')

# 1. Load the Data

In [4]:
# 다운로드 및 field확인
nltk.download('conll2002')
nltk.corpus.conll2002.fileids()

[nltk_data] Downloading package conll2002 to /Users/mac/nltk_data...
[nltk_data]   Unzipping corpora/conll2002.zip.


['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']

In [6]:
data = nltk.corpus.conll2002
train_sents =  list(data.iob_sents('esp.train'))
test_sents = list(data.iob_sents('esp.testb'))

In [8]:
# column : token | pos_tag | ner_tag
train_sents[0]

[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

# 2. using PyCRFSuite 
## PyCRFSuite란?
- c++로 구현된 CRFSuite 구현체를 Python 환경에서 이용할 수 있도록 도와주는 라이브러리
- 해당 라이브러리를 이용하기위해서는 potential function을 직접 디자인해야 함

### potential function
- word2features는 문장 sent의 시점 i에 대한 potential function임

In [107]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word.lower=' + word.lower(), # word lower
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(), # word is upper?
        'word.istitle=%s' % word.istitle(), # word is title?
        'word.isdigit=%s' % word.isdigit(), # word is digit?
        'postag=' + postag,
        'postag[:2]=' + postag[:2],
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            '-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
    if i < len(sent) - 1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            '+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features

### Sentence to features, labels, tokens 

In [108]:
def sent2features(sent):
    return [word2features(sent, i) for  i in range(len(sent))]
def sent2labels(sent):
    return [label for token, postag, label in sent]
def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [109]:
sent2features(train_sents[0])[0]

['bias',
 'word.lower=melbourne',
 'word[-3:]=rne',
 'word[-2:]=ne',
 'word.isupper=False',
 'word.istitle=True',
 'word.isdigit=False',
 'postag=NP',
 'postag[:2]=NP',
 'BOS',
 '+1:word.lower=(',
 '+1:word.istitle=False',
 '+1:word.isupper=False',
 '+1:postag=Fpa',
 '+1:postag[:2]=Fp']

In [110]:
# 문장을 학습 가능한 형태의 데이터로 변환
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

# 3. Train the model
## 3-1. 주어진 모든 feature를 다 가지고 학습

In [111]:
# 모델에 데이터를 append 하여 학습할 준비 함
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq,yseq)

In [112]:
# parameter setting
# 최소 다섯번 이상 등장한 feature만 이용
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True,
    
    # minimum frequency
    'feature.minfreq': 5
})

In [113]:
# 모델 학습
data_name = 'conll2002-esp.crfsuite'
trainer.train(data_name)

In [114]:
tagger = pycrfsuite.Tagger() # 학습된 모델을 tagger로 불러옴
tagger.open(data_name)

In [115]:
# 테스트 문장에 대하여 ner tagging 수행
ex_sent = test_sents[0]
print(' '.join(sent2tokens(ex_sent)), end='\n\n')
print("Predicted:",', '.join(tagger.tag(sent2features(ex_sent))))
print("Correct:",', '.join(sent2labels(ex_sent)))

La Coruña , 23 may ( EFECOM ) .

Predicted: B-LOC, I-LOC, O, O, O, O, B-ORG, O, O
Correct: B-LOC, I-LOC, O, O, O, O, B-ORG, O, O


In [116]:
# for tagging performance
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = preprocessing.LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
        
    )

In [117]:
y_true = y_test
y_pred = []
for sent in test_sents:
    y_pred.append(tagger.tag(sent2features(sent)))

In [118]:
bio_classification_report(y_true, y_pred)

'              precision    recall  f1-score   support\n\n       B-LOC       0.74      0.71      0.73      1084\n       I-LOC       0.57      0.51      0.54       325\n      B-MISC       0.61      0.37      0.46       339\n      I-MISC       0.59      0.43      0.50       557\n       B-ORG       0.76      0.78      0.77      1400\n       I-ORG       0.78      0.76      0.77      1104\n       B-PER       0.77      0.87      0.82       735\n       I-PER       0.83      0.94      0.88       634\n\n   micro avg       0.75      0.72      0.73      6178\n   macro avg       0.71      0.67      0.68      6178\nweighted avg       0.74      0.72      0.73      6178\n samples avg       0.09      0.09      0.09      6178\n'

## 3-2. 한정된 feature만 가지고 학습
- bias, word lower, word[-3:], word[-2:]만 이용

In [97]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
        ])
    else:
        features.append('EOS')
                
    return features


In [98]:
# 문장을 학습 가능한 형태의 데이터로 변환
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

In [100]:
# 모델에 데이터를 append 하여 학습할 준비 함
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq,yseq)

In [101]:
# parameter setting
# 최소 다섯번 이상 등장한 feature만 이용
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True,
    
    # minimum frequency
    'feature.minfreq': 5
})

In [102]:
# 모델 학습
data_name = 'conll2002-esp.crfsuite'
trainer.train(data_name)
tagger = pycrfsuite.Tagger() # 학습된 모델을 tagger로 불러옴
tagger.open(data_name)

In [104]:
y_true = y_test
y_pred = []
for sent in test_sents:
    y_pred.append(tagger.tag(sent2features(sent)))

In [105]:
bio_classification_report(y_true, y_pred)

'              precision    recall  f1-score   support\n\n       B-LOC       0.69      0.49      0.58      1084\n       I-LOC       0.60      0.47      0.52       325\n      B-MISC       0.52      0.20      0.29       339\n      I-MISC       0.52      0.36      0.43       557\n       B-ORG       0.74      0.55      0.63      1400\n       I-ORG       0.71      0.52      0.60      1104\n       B-PER       0.83      0.69      0.76       735\n       I-PER       0.86      0.86      0.86       634\n\n   micro avg       0.72      0.54      0.62      6178\n   macro avg       0.68      0.52      0.58      6178\nweighted avg       0.71      0.54      0.61      6178\n samples avg       0.07      0.07      0.07      6178\n'

# 4.모델 확인
- 영향력이 높은 features, 각각에 해당하는 weight확인
- 3-1에서 모든 feature를 이용했던 모델로 평가
- 해당 결과로 ner tagging에서 중요한 정보는 앞/뒤에 등장하는 단어임을 알 수 있음

In [119]:
debugger = tagger.info()
weights = debugger.state_features
location_features = {feature:weight for feature, weight in weights.items() if 'LOC' in feature[1]}

for feature, weight in sorted(location_features.items(), key=lambda x:-x[1])[:50]:
    print('{} : {}'.format(feature, weight))

('-1:word.lower=nuboso', 'B-LOC') : 4.041605
('word.lower=líbano', 'B-LOC') : 3.074029
('-1:word.lower=calle', 'I-LOC') : 2.933811
('word.lower=vitoria', 'B-LOC') : 2.886962
('-1:word.lower=despejado', 'B-LOC') : 2.829584
('-1:word.lower=cantabria', 'B-LOC') : 2.817601
('word.lower=pamplona', 'B-LOC') : 2.659921
('-1:word.lower=santa', 'I-LOC') : 2.575717
('-1:word.lower=desde', 'B-LOC') : 2.571576
('word.lower=londres', 'B-LOC') : 2.439067
('word.lower=melilla', 'B-LOC') : 2.377884
('word.lower=palacio', 'B-LOC') : 2.289678
('word.lower=cáceres', 'B-LOC') : 2.284109
('word.lower=bruselas', 'B-LOC') : 2.252099
('+1:word.lower=coruña', 'B-LOC') : 2.143314
('word.lower=cantabria', 'B-LOC') : 2.088536
('word.lower=badajoz', 'B-LOC') : 2.074211
('word.lower=santander', 'B-LOC') : 2.048165
('-1:word.lower=plaza', 'I-LOC') : 2.024622
('-1:word.lower=en', 'B-LOC') : 1.979929
('-1:word.lower=de', 'I-LOC') : 1.978107
('word.lower=estadio', 'B-LOC') : 1.977449
('-1:word.lower=ciudad', 'I-LOC') :